In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'gender_submission.csv']


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
!pip install fastai==0.7.0

In [4]:
from fastai.imports import *
from fastai.structured import *

#from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [5]:
PATH = '../input'
!ls {PATH}

gender_submission.csv  test.csv  train.csv


In [6]:
df_raw = pd.read_csv(f'{PATH}/train.csv', low_memory = False)

In [24]:
df_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_len,hasCabin,familySize
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2,23,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,51,1,2
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,22,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2,44,1,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2,24,0,1


In [27]:
df_raw.isnull().sum().sort_index()/len(df_raw)

Age            0.000000
Cabin          0.771044
Embarked       0.000000
Fare           0.000000
Name           0.000000
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
familySize     0.000000
hasCabin       0.000000
name_len       0.000000
dtype: float64

In [9]:
#Let's get the length of the name
df_raw['name_len'] = df_raw['Name'].apply(len)
df_raw['hasCabin'] = df_raw['Cabin'].apply(lambda x: 0 if type(x) == float else 1)  #reverse this value and check your score again

In [10]:
#Let's check the datatype of the columns
df_raw.shape

(891, 14)

In [11]:
#new feature as a family size
#for index,row in df_raw.iterrows():
#    row['familySize'] = row['SibSp'] + row['Parch'] + 1

df_raw["familySize"] = df_raw.apply(lambda row: row["SibSp"] + row["Parch"] + 1, axis = 1)

In [12]:
#Feature to identifying alone travellers
#for row in df_raw:
#    row["isAlone"] = 0
#    row.loc[row["familySize"] == 1, "isAlone"] = 1

#df_raw["isAlone"] = df_raw.apply(lambda row: 1 if row["familySize"] == 1 else 0)


In [16]:
#df_raw['Name'].iloc[0]
#remove nulls from embarked column
#for ind in df_raw.index:
#    #print(ind)
#    df_raw['Embarked'].iloc[ind] = df_raw['Embarked'].iloc[ind].fillna('S')

#values = {'Embarked' : 'S'}

df_raw["Embarked"] = df_raw["Embarked"].fillna('S')


In [18]:
# convert string categorical variables to category data type.
train_cats(df_raw)

In [19]:
df_raw.Embarked.cat.categories

Index(['C', 'Q', 'S'], dtype='object')

In [20]:
df_raw.Embarked = df_raw.Embarked.cat.codes

In [22]:
df_raw.Sex.cat.categories

Index(['female', 'male'], dtype='object')

In [23]:
df_raw.Sex = df_raw.Sex.cat.codes

In [32]:
#let's drop the column with textual data. Later try dropping parch and SibSp as well, as we have already considered the family count
df_raw.drop(["Name", "Ticket", "PassengerId", "Cabin"], axis = 1)   

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,name_len,hasCabin,familySize
0,0,3,1,22,1,0,7,2,23,0,2
1,1,1,0,38,1,0,71,0,51,1,2
2,1,3,0,26,0,0,7,2,22,0,1
3,1,1,0,35,1,0,53,2,44,1,2
4,0,3,1,35,0,0,8,2,24,0,1
5,0,3,1,29,0,0,8,1,16,0,1
6,0,1,1,54,0,0,51,2,23,1,1
7,0,3,1,2,3,1,21,2,30,0,5
8,1,3,0,27,0,2,11,2,49,0,3
9,1,2,0,14,1,0,30,0,35,0,2


In [31]:
# Work on Age, Ticket, Fare and Cabin is still pending
# Let's fill missing values in Age column with a column average 
df_raw["Age"].fillna(df_raw["Age"].mean(), inplace = True)
df_raw["Age"] = df_raw["Age"].astype(int)

In [28]:
# Convert Fare from float to int
df_raw["Fare"] = df_raw["Fare"].astype(int)

In [33]:
# Preparing data for the model
df, y, nas = proc_df(df_raw, 'Survived')

In [35]:
# Let's now try to fit the model

m = RandomForestClassifier(n_jobs = -1)
m.fit(df, y)
m.score(df, y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9887766554433222

In [38]:
def split_vals(a,n):
    return a[:n].copy(), a[n:].copy()

n_valid = 418 #same as Kaggle's test data size
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((473, 14), (473,), (418, 14), (418,))

In [41]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid), 
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'):
        res.append(m.oob_score_)
    print(res)

In [42]:
m = RandomForestClassifier(n_jobs = -1)
%time m.fit(X_train, y_train)
print_score(m)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 24 ms, sys: 28 ms, total: 52 ms
Wall time: 111 ms
[0.14540168172557633, 0.4482822077654243, 0.9788583509513742, 0.7990430622009569]
